# Tire Degradation Prediction Model - Week 5

## Overview
This notebook implements advanced models to predict tire degradation in Formula 1 races. Since tire degradation is not strictly linear and depends on multiple factors (compound type, track temperature, driving style, etc.), we'll use sequence models like LSTM to capture these complex patterns.

## Approach
1. **Data Exploration**
   - Analyze the relationship between lap times and tire age
   - Visualize performance degradation patterns by compound
   - Determine how to quantify "degradation" (lap time delta or derived metric)

2. **Feature Engineering**
   - Create a derived tire degradation metric
   - Organize data into sequential format for LSTM
   - Normalize features appropriately
   - Create sliding windows of previous laps to predict future performance

3. **Model Development**
   - **Primary Model**: LSTM network to predict degradation trajectory
   - **Alternative Model**: XGBoost with quantile regression for uncertainty estimation

4. **Evaluation & Visualization**
   - Compare predicted vs. actual degradation curves
   - Analyze performance across different compounds and race conditions
   - Create interactive Plotly visualizations of degradation patterns

5. **Implementation Details**
   - Sequence length: 5 laps (input) → predict next 3-5 laps
   - Features: Tire age, compound, lap time trends, fuel load
   - Target: Derived degradation metric or direct lap time prediction

## Expected Outcomes
- Trained model to predict tire performance over extended stints
- Uncertainty bounds for degradation predictions (10th, 50th, 90th percentiles)
- Interactive visualization of degradation curves by compound

---

## 1. Importing Necessary Libraries and Creatind New Directories

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
# Import utility functions from our module
from ML_utils.N00_model_lap_prediction import compound_colors, compound_names

# Set plot style
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Create output directories if they don't exist
os.makedirs('../../outputs/week5/models', exist_ok=True)


---

## 2. Loading Dataframes

In [ ]:
data = pd.read_csv("../../outputs/week3/lap_prediction_data.csv")
print("\nRegular data sample:")
display(data.head())

In [ ]:
seq_data = pd.read_csv("../../outputs/week3/sequential_lap_prediction_data.csv")
print("\nSequential data sample:")
display(seq_data.head())

In [ ]:
# Display basic information
print("Basic dataset information:")
print(f"Regular data shape: {data.shape}")
print(f"Sequential data shape: {seq_data.shape}")

---

## 3. Locating Tire Related Columns


In [ ]:
# Check for tire-related columns
tire_columns = ['CompoundID', 'TyreAge']
print(f"\nTire-related columns: {tire_columns}")

In [ ]:
# Summary statistics for tire-related columns
print("\nTire-related statistics:")
display(data[tire_columns].describe())

---

## 4. Compound Mappings

In [ ]:
# Print compound mappings for reference
print("\nCompound mappings:")
print(f"Compound names: {compound_names}")
print(f"Compound colors: {compound_colors}")

---

## 5. Analyzing Relationship between Tire Age and Lap Time by compound

In [ ]:
# Encapsulate the tire-degradation-by-compound plot into a function
def plot_tire_degradation_by_compound(
    data,
    compound_colors,
    compound_names,
    save_path=None
):
    """
    Plot lap time vs tire age for each compound, with error bands,
    and optionally save the figure.

    Args:
        data (pd.DataFrame): must contain 'CompoundID', 'TyreAge', 'LapTime'
        compound_colors (dict): mapping CompoundID -> color string
        compound_names (dict): mapping CompoundID -> display name
        save_path (str or Path, optional): filepath to save the figure

    Returns:
        matplotlib.figure.Figure: the generated figure
    """
    import matplotlib.pyplot as plt

    # Create figure
    fig, ax = plt.subplots(figsize=(12, 6))

    # Loop over each compound
    for compound_id in data['CompoundID'].unique():
        subset = data[data['CompoundID'] == compound_id]
        agg = (
            subset
            .groupby('TyreAge')['LapTime']
            .agg(['mean', 'std', 'count'])
            .reset_index()
        )

        # Only plot if there's more than one point
        if len(agg) > 1:
            color = compound_colors.get(compound_id, 'black')
            name = compound_names.get(compound_id, f'Unknown ({compound_id})')

            ax.plot(
                agg['TyreAge'],
                agg['mean'],
                'o-',
                color=color,
                label=f'{name} Tire'
            )
            # Error band
            ax.fill_between(
                agg['TyreAge'],
                agg['mean'] - agg['std'],
                agg['mean'] + agg['std'],
                color=color,
                alpha=0.2
            )

    ax.set_xlabel('Tire Age (laps)')
    ax.set_ylabel('Lap Time (s)')
    ax.set_title('Tire Degradation: Effect on Lap Time')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Save if requested
    if save_path:
        fig.savefig(save_path)

    plt.show()
    return fig

fig_tire_curve = plot_tire_degradation_by_compound(
    data,
    compound_colors,
    compound_names,
    save_path='../../outputs/week5/tire_deg_curve.png'
)


---

## 6. Exploring Lap Times Deltas and Tire Ages

In [ ]:
# Encapsulate lap time delta exploration into a function
def plot_lap_time_deltas_by_tyre_age(
    seq_data,
    compound_colors,
    compound_names,
    save_path=None
):
    """
    Plot average LapTime_Delta vs TyreAge for each compound.
    Positive delta means lap times are getting slower.

    Args:
        seq_data (pd.DataFrame): must contain 'CompoundID', 'TyreAge', 'LapTime_Delta'
        compound_colors (dict): mapping CompoundID -> color
        compound_names (dict): mapping CompoundID -> display name
        save_path (str or Path, optional): path to save the figure

    Returns:
        matplotlib.figure.Figure: the generated figure
    """
    import matplotlib.pyplot as plt

    if 'LapTime_Delta' not in seq_data.columns:
        print("LapTime_Delta column not available in the data.")
        return None

    fig, ax = plt.subplots(figsize=(12, 6))

    # Loop through compounds
    for compound_id in seq_data['CompoundID'].unique():
        subset = seq_data[seq_data['CompoundID'] == compound_id]
        agg = subset.groupby('TyreAge')['LapTime_Delta'].mean().reset_index()

        if len(agg) > 1:
            color = compound_colors.get(compound_id, 'black')
            name = compound_names.get(compound_id, f'Unknown ({compound_id})')
            ax.plot(
                agg['TyreAge'],
                agg['LapTime_Delta'],
                'o-',
                color=color,
                label=f'{name} Tire'
            )

    ax.axhline(y=0, linestyle='--', color='black', alpha=0.5)
    ax.set_xlabel('Tire Age (laps)')
    ax.set_ylabel('Lap Time Delta (s) - Positive means getting slower')
    ax.set_title('Lap Time Degradation Rate by Tire Age')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Save figure if a path is provided
    if save_path:
        fig.savefig(save_path)

    plt.tight_layout()
    plt.show()
    return fig

# Call the function on your DataFrame
fig_lap_delta = plot_lap_time_deltas_by_tyre_age(
    seq_data,
    compound_colors,
    compound_names,
    save_path='../../outputs/week5/tire_deg_rate.png'
)


---

## 7. Exploring if Tire Age affects Speed in different Sectors

In [ ]:
# Encapsulate the tire-age speed effect plot into a function
def plot_tire_age_speed_effect(
    data,
    compound_id=2,
    compound_names=None,
    speed_columns=None,
    save_path=None
):
    """
    Plot how tire age affects speed in specified sectors for a given compound.

    Args:
        data (pd.DataFrame): must contain 'CompoundID', 'TyreAge' and the speed columns
        compound_id (int): the compound to focus on
        compound_names (dict): mapping CompoundID -> display name
        speed_columns (list of str): list of speed column names to plot
        save_path (str or Path, optional): filepath to save the figure

    Returns:
        matplotlib.figure.Figure: the generated figure
    """
    import matplotlib.pyplot as plt

    if speed_columns is None:
        speed_columns = ['SpeedI1', 'SpeedI2', 'SpeedFL']
    if compound_names is None:
        compound_display = str(compound_id)
    else:
        compound_display = compound_names.get(compound_id, f'Compound {compound_id}')

    # Filter to the chosen compound
    subset = data[data['CompoundID'] == compound_id]

    # Create figure
    fig, ax = plt.subplots(figsize=(14, 8))

    # Plot each speed column
    for speed_col in speed_columns:
        if speed_col not in subset.columns:
            continue
        agg = subset.groupby('TyreAge')[speed_col].mean().reset_index()
        if len(agg) > 1:
            ax.plot(
                agg['TyreAge'],
                agg[speed_col],
                'o-',
                label=speed_col
            )

    ax.set_xlabel('Tire Age (laps)')
    ax.set_ylabel('Speed (kph)')
    ax.set_title(f'Effect of Tire Age on Speed - {compound_display} Tires')
    ax.legend()
    ax.grid(True, alpha=0.3)
    fig.tight_layout()

    # Save if requested
    if save_path:
        fig.savefig(save_path)

    plt.show()
    return fig

# Call the function on your DataFrame
fig_speed_effect = plot_tire_age_speed_effect(
    data,
    compound_id=2,
    compound_names=compound_names,
    speed_columns=['SpeedI1', 'SpeedI2', 'SpeedFL'],
    save_path='../../outputs/week5/tire_age_speed_effect.png'
)


---

## 8. Creating Tire Degradation Metrics

For being able to predict tyre degradation more effectively, a good option can be generate more variables with the current data that we have. Therefore, I´ll add this new variables:

## Disclaimer: importance of fuel load.

After making the cells and looking at the data, some tires shows positive degradation. That means that lap times are descending instead of going up. This is caused due to the less amount of fuel during the race. Therefore, I need to **create an adjusted lap time** that takes into account this fuel factor before creating our prediction models. 

Then, I will create this variable and then adjust the plots and variable calculation for fitting this feature.

*NOTE*: fuel burn calculation and impact will be calculated according to this articles: 

- [BBC Sport Weight Reduction](https://www.bbc.com/sport/articles/cv2g715dkk1o#:~:text=A%201.5kg%20reduction%20in,so%20over%20a%20race%20distance.)

- [Fuel Correction Analysis, Medium](https://medium.com/@umakschually/fuel-correction-29ccd98ae62b#:~:text=Rule%20of%20thumb%20is%20that,tyre%20age%20or%20anything%20else.)

#### 1. Absolute Tire Degradation (TireDegAbsolute)

Its objective is to **measure how much seconds is the actual lap time slower, compared with the baseline** (new tires or with the less degradation possible, for instance, only 2 laps).

**Positive values** implie degradation (car is getting slower). As I said, it would be measured in **seconds**.

- *Utility*:
    - Allows knowing the direct impact on lap time.
    - Helps to determine the *cross point* when a pit stop becomes an advantage.
    - Fundamental for strategic calculus, as teams work with absolute times.
    - Helps us answering the following: **How many seconds are we losing per lap with degradation?**

#### 2. Tire Degradation Percentage 

It expresses degradation as an **augmenting percentage** to base time. For instance, 2% means the car is 2% slower than with new tires.

- *Utility*:
    - Allows more intuitive comparisons between different conditions.
    - Normalizes the data for more clear comparisons between tires.
    - Helps us aswering the following:**Which compound maintains better its relative performance?**

#### 3. Degradation Rate

Means how much time increases per lap with each aditional lap. Represents the first derivative of degradation curve. 

- *Utility:*
    - Allows knowing if degradation is lineal, progressive or if it stabilizes.
    - Crucial for estimating optimum pit stop window during races.
    - Allows anticipating future compound´s behaviour.
    - Helps us answering the following: **Degradation is getting worse or it is stabilizing?**.



---

### Calculating Lap Time Improvement Per Lap

In [ ]:
# Crear métricas de degradación ajustadas usando directamente la mejora de tiempo por vuelta
# Tiempo que mejora cada vuelta debido a la reducción de combustible
LAP_TIME_IMPROVEMENT_PER_LAP = 0.055  # segundos por vuelta (punto medio de 0.05-0.06s)

# Create a DataFrame to store all results with fuel adjustment
tire_deg_data = pd.DataFrame()

# Process each compound separately
for compound_id in data['CompoundID'].unique():
    compound_name = compound_names.get(compound_id, f"Unknown ({compound_id})")
    print(f"Processing {compound_name} tires (ID: {compound_id})...")
    
    # Filter for this compound
    compound_data = data[data['CompoundID'] == compound_id].copy()
    
    # Sort by TyreAge to see the degradation trend
    compound_data = compound_data.sort_values('TyreAge')
    
    # Check if we have enough data
    if len(compound_data) < 5:
        print(f"  Not enough data for {compound_name} tires, skipping")
        continue
    
    # Find baseline information
    if 1 in compound_data['TyreAge'].values:
        # Get baseline data (TyreAge=1)
        baseline_data = compound_data[compound_data['TyreAge'] == 1]
        baseline_lap_time = baseline_data['LapTime'].mean()
        baseline_tire_age = 1
    else:
        # If no 'new tire' laps, use the minimum TyreAge available
        min_age = compound_data['TyreAge'].min()
        baseline_data = compound_data[compound_data['TyreAge'] == min_age]
        baseline_lap_time = baseline_data['LapTime'].mean()
        baseline_tire_age = min_age
        print(f"  No laps with new tires for {compound_name}, using TyreAge={min_age} as baseline")
    
    # Calculate fuel adjustment directly based on laps from baseline
    compound_data['LapsFromBaseline'] = compound_data['TyreAge'] - baseline_tire_age
    compound_data['FuelEffect'] = compound_data['LapsFromBaseline'] * LAP_TIME_IMPROVEMENT_PER_LAP
    
    # Calculate fuel-adjusted lap time
    compound_data['FuelAdjustedLapTime'] = compound_data['LapTime'] + compound_data['FuelEffect']
    
    # Calculate traditional degradation metrics
    compound_data['TireDegAbsolute'] = compound_data['LapTime'] - baseline_lap_time
    compound_data['TireDegPercent'] = (compound_data['LapTime'] / baseline_lap_time - 1) * 100
    
    # Calculate fuel-adjusted degradation metrics
    baseline_adjusted_lap_time = baseline_lap_time  # For new tires, no adjustment needed
    compound_data['FuelAdjustedDegAbsolute'] = compound_data['FuelAdjustedLapTime'] - baseline_adjusted_lap_time
    compound_data['FuelAdjustedDegPercent'] = (compound_data['FuelAdjustedLapTime'] / baseline_adjusted_lap_time - 1) * 100
    
    # Add compound info for later aggregation
    compound_data['CompoundName'] = compound_name
    
    # Add to the combined DataFrame
    tire_deg_data = pd.concat([tire_deg_data, compound_data])
    
    # Calculate maximum laps and total fuel effect
    max_laps = compound_data['TyreAge'].max() - baseline_tire_age
    total_fuel_effect = max_laps * LAP_TIME_IMPROVEMENT_PER_LAP
    
    print(f"  Baseline lap time for {compound_name}: {baseline_lap_time:.3f}s")
    print(f"  Maximum laps from baseline: {max_laps:.0f}")
    print(f"  Estimated total fuel benefit: ~{total_fuel_effect:.2f}s")
    print(f"  Processed {len(compound_data)} laps with {compound_name} tires")

# Display comparison between regular and fuel-adjusted metrics
print("\nComparison of regular vs. fuel-adjusted metrics (sample):")
sample_comparison = tire_deg_data.groupby(['CompoundName', 'TyreAge'])[
    ['TireDegAbsolute', 'FuelAdjustedDegAbsolute', 'FuelEffect']
].mean().reset_index()
display(sample_comparison.head(10))


---

### Plotting the Diferrence between regular and Fuel Adjusted Degradation

In [ ]:
def plot_regular_vs_adjusted_degradation(tire_deg_data, compound_names, compound_colors, LAP_TIME_IMPROVEMENT_PER_LAP):
    # Create a comparison of regular vs fuel-adjusted degradation
    plt.figure(figsize=(16, 12))
    compound_ids = tire_deg_data['CompoundID'].unique()
    
    # Loop through the compounds to create comparison plots
    for i, compound_id in enumerate(compound_ids):
        compound_subset = tire_deg_data[tire_deg_data['CompoundID'] == compound_id]
        color = compound_colors.get(compound_id, 'black')
        compound_name = compound_names.get(compound_id, f'Unknown ({compound_id})')
        
        # Calculate means for regular and adjusted degradation
        reg_agg = compound_subset.groupby('TyreAge')['TireDegAbsolute'].mean()
        adj_agg = compound_subset.groupby('TyreAge')['FuelAdjustedDegAbsolute'].mean()
        
        # Create subplot
        plt.subplot(len(compound_ids), 1, i+1)
        
        # Plot regular degradation
        plt.plot(reg_agg.index, reg_agg.values, 'o--', 
                 color=color, alpha=0.5, label=f'{compound_name} (Regular)')
        
        # Plot fuel-adjusted degradation
        plt.plot(adj_agg.index, adj_agg.values, 'o-', 
                 color=color, linewidth=2, label=f'{compound_name} (Fuel Adjusted)')
        
        plt.axhline(y=0, color='gray', linestyle='--', alpha=0.7)
        plt.ylabel('Degradation (s)')
        plt.title(f'{compound_name} Tire Degradation: Regular vs. Fuel-Adjusted')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        
        min_lap = reg_agg.index.min()
        max_lap = reg_agg.index.max()
        total_laps = max_lap - min_lap
        total_fuel_effect = total_laps * LAP_TIME_IMPROVEMENT_PER_LAP
        plt.annotate(f"Est. total fuel effect: ~{total_fuel_effect:.2f}s", 
                     xy=(0.02, 0.05), xycoords='axes fraction',
                     bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8))
        
        if i == len(compound_ids)-1:  
            plt.xlabel('Tire Age (laps)')

    plt.tight_layout()
    plt.savefig('../../outputs/week5/regular_vs_adjusted_comparison.png')
    plt.show()

plot_regular_vs_adjusted_degradation(tire_deg_data, compound_names, compound_colors, LAP_TIME_IMPROVEMENT_PER_LAP)


---

### 8.1 Absolute Tire Degradation

In [ ]:
def plot_fuel_adjusted_degradation(tire_deg_data, compound_names, compound_colors):
    # Visualize the fuel-adjusted absolute degradation
    plt.figure(figsize=(14, 7))
    compound_ids = tire_deg_data['CompoundID'].unique()

    for compound_id in compound_ids:
        compound_subset = tire_deg_data[tire_deg_data['CompoundID'] == compound_id]
        color = compound_colors.get(compound_id, 'black')
        compound_name = compound_names.get(compound_id, f'Unknown ({compound_id})')
        
        # Aggregate data for line plot
        agg_data = compound_subset.groupby('TyreAge')['FuelAdjustedDegAbsolute'].agg(['mean', 'std']).reset_index()
        
        # Plot mean line
        plt.plot(agg_data['TyreAge'], agg_data['mean'], 'o-', 
                 color=color, linewidth=2, label=f'{compound_name}')
        
        # Add error bands if we have standard deviation
        if 'std' in agg_data.columns and not agg_data['std'].isnull().all():
            plt.fill_between(agg_data['TyreAge'], 
                             agg_data['mean'] - agg_data['std'], 
                             agg_data['mean'] + agg_data['std'],
                             color=color, alpha=0.2)

    plt.axhline(y=0, color='gray', linestyle='--', alpha=0.7)
    plt.xlabel('Tire Age (laps)')
    plt.ylabel('Fuel-Adjusted Absolute Degradation (s)')
    plt.title('Tire Degradation by Compound and Age (Fuel Effect Removed)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('../../outputs/week5/fuel_adjusted_deg_by_compound.png')
    plt.show()

plot_fuel_adjusted_degradation(tire_deg_data, compound_names, compound_colors)


---

### 8.2 Tire Degradation Percentage

In [ ]:
def plot_fuel_adjusted_percentage_degradation(tire_deg_data, compound_names, compound_colors):
    # Visualize the fuel-adjusted percentage degradation
    plt.figure(figsize=(14, 7))

    compound_ids = tire_deg_data['CompoundID'].unique()

    for compound_id in compound_ids:
        compound_subset = tire_deg_data[tire_deg_data['CompoundID'] == compound_id]
        color = compound_colors.get(compound_id, 'black')
        compound_name = compound_names.get(compound_id, f'Unknown ({compound_id})')
        
        # Aggregate data for line plot
        agg_data = compound_subset.groupby('TyreAge')['FuelAdjustedDegPercent'].agg(['mean', 'std']).reset_index()
        
        # Plot mean line
        plt.plot(agg_data['TyreAge'], agg_data['mean'], 'o-', 
                 color=color, linewidth=2, label=f'{compound_name}')
        
        # Add error bands if we have standard deviation
        if 'std' in agg_data.columns and not agg_data['std'].isnull().all():
            plt.fill_between(agg_data['TyreAge'], 
                             agg_data['mean'] - agg_data['std'], 
                             agg_data['mean'] + agg_data['std'],
                             color=color, alpha=0.2)

    plt.axhline(y=0, color='gray', linestyle='--', alpha=0.7)
    plt.xlabel('Tire Age (laps)')
    plt.ylabel('Fuel-Adjusted Percentage Degradation (%)')
    plt.title('Percentage Tire Degradation by Compound and Age (Fuel Effect Removed)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('../../outputs/week5/fuel_adjusted_deg_percent_by_compound.png')
    plt.show()

plot_fuel_adjusted_percentage_degradation(tire_deg_data, compound_names, compound_colors)


---

### 8.3 Tire Degradation Rate

In [ ]:
compound_ids = tire_deg_data['CompoundID'].unique()

# After plotting, add the variable to the dataframe
for compound_id in compound_ids:
    # Recalculate using the same method as in the visualization
    compound_subset = tire_deg_data[tire_deg_data['CompoundID'] == compound_id]
    avg_laptimes = compound_subset.groupby('TyreAge')['FuelAdjustedLapTime'].mean()
    deg_rates = avg_laptimes.diff()
    
    # Assign values to the dataframe
    for age, rate in zip(deg_rates.index, deg_rates.values):
        mask = (tire_deg_data['CompoundID'] == compound_id) & (tire_deg_data['TyreAge'] == age)
        tire_deg_data.loc[mask, 'DegradationRate'] = rate

# Verify that it has been added correctly
print("\nFirst rows with DegradationRate:")
display(tire_deg_data[['CompoundID', 'TyreAge', 'FuelAdjustedLapTime', 'DegradationRate']].head(10))


In [ ]:
def plot_fuel_adjusted_degradation_rate(tire_deg_data, compound_names, compound_colors):
    # Plot a line chart showing Tire Degradation Rate by compound with error bands
    plt.figure(figsize=(14, 7))

    compound_ids = tire_deg_data['CompoundID'].unique()

    for compound_id in compound_ids:
        # Filter the data for the current compound
        compound_subset = tire_deg_data[tire_deg_data['CompoundID'] == compound_id]
        
        # Calculate the average and standard deviation of degradation rate per tire age
        deg_stats = compound_subset.groupby('TyreAge')['DegradationRate'].agg(['mean', 'std']).reset_index()
        
        # Get color and compound name for the plot
        color = compound_colors.get(compound_id, 'black')
        compound_name = compound_names.get(compound_id, f'Unknown ({compound_id})')
        
        # Plot the line for this compound
        plt.plot(deg_stats['TyreAge'], deg_stats['mean'], marker='o', linestyle='-',
                 color=color, linewidth=2, label=compound_name)
        
        # Add error bands (standard deviation)
        if 'std' in deg_stats.columns and not deg_stats['std'].isnull().all():
            plt.fill_between(deg_stats['TyreAge'], 
                             deg_stats['mean'] - deg_stats['std'], 
                             deg_stats['mean'] + deg_stats['std'],
                             color=color, alpha=0.2)

    plt.axhline(y=0, color='gray', linestyle='--', alpha=0.7)
    plt.xlabel('Tire Age (laps)')
    plt.ylabel('Fuel-Adjusted Degradation Rate (s/lap)')
    plt.title('Tire Degradation Rate by Compound (Fuel Effect Removed)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()


plot_fuel_adjusted_degradation_rate(tire_deg_data, compound_names, compound_colors)


In [ ]:
# Fixing NaNs. This are due to diff pandas method. Assume 0 as new tires do not have degradation still

num_nans = tire_deg_data['DegradationRate'].isna().sum()
print(num_nans)

tire_deg_data['DegradationRate'] = tire_deg_data['DegradationRate'].fillna(0)
print(f"Number of NaN after sustitution: {tire_deg_data['DegradationRate'].isna().sum()}")

## Key Findings

#### Medium Tire (Yellow).

- The fuel effect was masking significantly degradation. With the adjustment, mroe degradation can be seen.

- They offer the best initial advantage (-4 seconds), that stabilizes in -3 seconds until aproximately lap 30.

- Total fuel impact is about 1.73 seconds faster at the end of the stint.

- They represent the best balance between performance and durability.

#### Hard Tire (Gray)

- Fuel effect made an stabilization ilusion and even improvement. The adjust reveals a constant and progressive degradation that goes up to +2seconds.

- Fuel effect on this tires are the biggest, with 2.64 fastet.

- Degradation rate is more stable and predictable, making them ideal for long stints.

#### Soft Tire (Red)

- There is a bigger volatile effect that it seemed without fuel effect.
- Erratic behaviour and big fluctuations after lap 20.
- Highly unpredictable and dramatically fluctating, specially after lap 20, with extreme degradation peaks of +2 seconds slower per lap.

### Detected Turning Points

- *SOFT*: show a cliff degradation over 20 laps.
- *MEDIUM*: change in pattern near lap 30.
- *HARD*: show a degradation increase after lap 40.


### Good Conclussions for predictive model. 

1. Fuel adjustment was essential for identifying true degradation.
2. Each compound shows unique patterns that can be useful for the model:
    - Soft tire as high volatile with critic points of sudden degradation.
    - Medium tire has a fast fall followed by stabilization.
    - Hard tire with slow but continous degradation.

3. Identified turning points are going to be crucial parameters for the LSTM or XgBoost, as they show critic moments for pit stop strategies.

---

## 9. Correlation Analysis: Tire-Related Factors with Lap Time

In [ ]:
# Invert dictionary to apply conversion to compound names to numbers 
compound_names_inv = {value: key for key, value in compound_names.items()}
# Replace the names with its according numbers
tire_deg_data["CompoundName"] = tire_deg_data["CompoundName"].replace(compound_names_inv)

# We can eliminate this column as it does not provide any information
tire_deg_data = tire_deg_data.drop('Unnamed: 0', axis=1)


In [ ]:
# Making correlation matrix
correlation_matrix = tire_deg_data.corr()


# Crear un heatmap
plt.figure(figsize=(24,12))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

---

## 10. Conclussions and Variable Cleaning

**Variables to Keep**  
- **FuelAdjustedLapTime** (remove `LapTime`, correlation 0.95)  
  - Main lap time metric adjusted without the misleading fuel effect  
- **FuelAdjustedDegPercent** (remove other degradation metrics)  
  - Best metric for comparing compounds  
  - The four degradation metrics have very high correlations with each other (0.98–1.00). The pther 3 variables were only created to add more explicability to the data analysis.
- **DegradationRate**  
  - Captures the changing dynamics of degradation  
  - Its low correlation with other variables confirms it adds unique information  
  - Crucial for detecting inflection points and sudden changes  
- **TyreAge**  
  - A fundamental variable for the model  
  - Tire age is the main predictor of its condition  
- **CompoundID** (remove `CompoundName`)  
  - Needed to distinguish between different compounds  

- **Rest of variables**

**Optional Variables (if they improve the model)**  
- **SpeedI1**, **SpeedI2**, **SpeedFL**: to capture sector effects  
- **FuelLoad**: as a control variable  

**Variables to Remove**  
- **LapTime** (use only `FuelAdjustedLapTime`)  
- **TireDegAbsolute**, **TireDegPercent**, **FuelAdjustedDegAbsolute**  
- **CompoundName** (redundant with `CompoundID`)  
- **LapsFromBaseline** and **FuelEffect** (perfect correlation)  

In [ ]:
# Remove only the specified redundant variables
columns_to_remove = [
    'LapTime',                 # Use only FuelAdjustedLapTime
    'TireDegAbsolute',         # Redundant with FuelAdjustedDegPercent
    'TireDegPercent',          # Redundant with FuelAdjustedDegPercent
    'FuelAdjustedDegAbsolute', # Redundant with FuelAdjustedDegPercent
    'CompoundName',            # Redundant with CompoundID
    'LapsFromBaseline',        # Perfect correlation
    'FuelEffect'               # Perfect correlation
]

# Drop columns from the dataframe
tire_deg_data = tire_deg_data.drop(columns=columns_to_remove)

# Show how many variables were removed and display a new correlation matrix
print(f"Removed {len(columns_to_remove)} redundant variables.")
print(f"The dataframe now has {tire_deg_data.shape[1]} columns.")




---

## 11. New Correlation Matrix to see interesting Variables

In [ ]:
# Display the new correlation matrix
updated_correlation_matrix = tire_deg_data.corr()
plt.figure(figsize=(16,8))
sns.heatmap(updated_correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Updated Correlation Matrix after Removing Redundant Variables")
plt.show()

After processing the data and applying fuel adjustment, the correlation matrix reveals critical relationships that will inform our LSTM model. Here are the main points:

- **Key Correlations:**
  - **FuelAdjustedLapTime & FuelAdjustedDegPercent:** Strong correlation (0.59) indicates similar degradation dynamics from different perspectives.
  - **TyreAge & LapsSincePitStop:** Almost perfect correlation (0.90), as both track similar information.
  - **CompoundID & FuelAdjustedDegPercent:** Moderate correlation (0.40) shows that tire compound significantly influences degradation.
  - **FuelLoad & Performance Metrics:** Strong negative correlation with Stint (-0.86) and moderate correlation with adjusted lap time (0.42), emphasizing the importance of fuel weight.
  - **DegradationRate:** Low correlation with most variables, including FuelAdjustedDegPercent (0.19), suggesting it captures unique degradation dynamics.

- **LSTM Model Input Selection:**
  - **Primary Variables:**
    - **FuelAdjustedLapTime:** Core performance metric without fuel effects.
    - **FuelAdjustedDegPercent:** Best metric for comparing compound performance.
    - **DegradationRate:** Captures lap-to-lap changes and critical inflection points.
    - **TyreAge:** Fundamental predictor of tire condition.
    - **CompoundID:** Essential for distinguishing between different tire compounds.
  - **Supporting Variables:**
    - **Speed Metrics (SpeedI1, SpeedI2, SpeedFL):** Capture sector-specific effects.
    - **FuelLoad:** Used as a control variable.
    - **Position:** Provides contextual race information.

---

## 12. Saving my final Dataframe as a CSV

In [ ]:

# Guardar el DataFrame procesado con las métricas de degradación ajustadas por combustible
output_path = "../../outputs/week5/tire_degradation_fuel_adjusted.csv"

# Guardar el DataFrame
tire_deg_data.to_csv(output_path, index=False)

---


## 13. Creating Sequential Data

In [ ]:
df = pd.read_csv("../../outputs/week5/tire_degradation_fuel_adjusted.csv")

## Sequencing Process for LSTM

### Creating Temporal Sequences:
- We need to transform our tabular data into chronologically ordered sequences.
- Each sequence should contain a sliding window of N consecutive laps (typically 5 laps).

### Data Format for LSTM:
**Input:** [lap_t-5, lap_t-4, lap_t-3, lap_t-2, lap_t-1]  

**Output:** [lap_t, lap_t+1, lap_t+2]

### Variables to Include in Each Sequence Element:
- FuelAdjustedLapTime
- FuelAdjustedDegPercent
- DegradationRate
- TyreAge
- CompoundID
- Contextual variables (FuelLoad, position, sector speeds)

It is crucial to ensure that the sequences maintain temporal integrity and do not mix data from different stints or pit stops.

In [ ]:
def create_sequences(df, input_length=5, prediction_horizon=3, target_column='DegradationRate'):
    """
    Create sequences for any sequential model (LSTM, TCN) from the tire degradation data.
    Groups by driver, stint and compound to ensure proper sequencing.
    
    Args:
        df: DataFrame with tire degradation data
        input_length: Number of consecutive laps to include in input sequence
        prediction_horizon: Number of future laps to predict
        target_column: Column to predict
        
    Returns:
        sequences: List of DataFrame sequences
        targets: List of target arrays
    """
    sequences = []
    targets = []
    
    # Group by DriverNumber, Stint, and CompoundID
    groupby_columns = ['DriverNumber', 'Stint', 'CompoundID']
    
    # Process each driver-stint-compound group separately
    for name, group in df.groupby(groupby_columns):
        # Sort by TyreAge to ensure chronological order
        sorted_group = group.sort_values('TyreAge').reset_index(drop=True)
        
        # Skip if we don't have enough laps for a sequence
        if len(sorted_group) < input_length + prediction_horizon:
            continue
        
        # Create sliding window sequences
        for i in range(len(sorted_group) - input_length - prediction_horizon + 1):
            # Get input sequence (all features)
            seq = sorted_group.iloc[i:i+input_length]
            
            # Get target values (future values to predict)
            target = sorted_group.iloc[i+input_length:i+input_length+prediction_horizon][target_column].values
            
            sequences.append(seq)
            targets.append(target)
    
    print(f"Created {len(sequences)} sequences of {input_length} laps each")
    return sequences, targets



In [ ]:
sequences, targets = create_sequences(df)

---

### Explanation of Sequences and Targets
#### What does the `create_sequences` function do?
The function creates data in a sequential format, necessary for training LSTM models. Specifically:

**Sequences:**
- They are "sliding windows" of consecutive data from the same set of tires.
- Each sequence contains data from 5 consecutive laps (all DataFrame columns).
- They represent the "recent history" that the model will use to make predictions.

**Targets:**
- These are the values we want to predict in the future.
- Each target contains the degradation values for the next 3 laps after the sequence.
- It only includes the column we want to predict (`FuelAdjustedDegPercent`).

#### Concrete Example
Imagine we have data from 10 laps with the same tire:

- **Sequence 1:** Laps 1-5
  - **Target 1:** Degradation in laps 6-8
- **Sequence 2:** Laps 2-6
  - **Target 2:** Degradation in laps 7-9
- **Sequence 3:** Laps 3-7
  - **Target 3:** Degradation in laps 8-10




---

### 13.1 New Approach: Divide sequences by compound ID

A good approach to try to improve the performance of our predictions is divide the data sequences by different tire compounds. In that way, I can try to develop 3 different models, each for every compound ID, alongside a global model. 

Like this, I can make the predictions using the specific model for that tire, compare it with the global prediction and try to do an emsemble between the two ones for improving robustness of the predictions


In [ ]:
# Divide sequences by compound tire
def split_by_compound(df, sequences, targets):
    """
    Divide sequences and targets per compound ID
    
    """
    # Identify unique tire id
    compounds = df['CompoundID'].unique()
    
    # Create dictionaries for storing sequences per compound
    compound_sequences = {c: [] for c in compounds}
    compound_targets = {c: [] for c in compounds}
    
    # Assign each sequence to the corresponend dictionary
    for i, seq in enumerate(sequences):
        compound = seq['CompoundID'].iloc[0]
        compound_sequences[compound].append(seq)
        compound_targets[compound].append(targets[i])
    
    # Informar sobre la distribución de secuencias
    for compound in compounds:
        compound_name = compound_names.get(compound, f"Compound {compound}")
        print(f"{compound_name}: {len(compound_sequences[compound])} sequences")
    
    return compound_sequences, compound_targets, compounds

In [ ]:
# Divide data per compound
compound_sequences, compound_targets, unique_compounds = split_by_compound(df, sequences, targets)

---

## 14. Verifying the Sequential Data

In [ ]:
def verify_sequences_with_targets(sequences, targets, num_to_check=3):
    print("COMPLETE SEQUENCE VERIFICATION (WITH TARGETS):")
    print("=============================================")
    
    # Check a few consecutive sequences from the same group
    driver_stint_compounds = []
    
    for i, seq in enumerate(sequences):
        # Get identifier for this sequence
        identifier = (seq['DriverNumber'].iloc[0], seq['Stint'].iloc[0], seq['CompoundID'].iloc[0])
        driver_stint_compounds.append(identifier)
    
    # Find groups with consecutive sequences
    for i in range(len(sequences)-1):
        # Check if consecutive sequences are from same driver-stint-compound
        if driver_stint_compounds[i] == driver_stint_compounds[i+1]:
            seq1 = sequences[i]
            seq2 = sequences[i+1]
            
            # Get tire ages and targets
            ages1 = seq1['TyreAge'].values
            ages2 = seq2['TyreAge'].values
            target1 = targets[i]
            target2 = targets[i+1]
            
            # Calculate what the next tire ages should be (for targets)
            expected_target_ages1 = np.array([ages1[-1] + j + 1 for j in range(len(target1))])
            expected_target_ages2 = np.array([ages2[-1] + j + 1 for j in range(len(target2))])
            
            # Check if sliding window pattern is correct
            sliding_window_correct = np.array_equal(ages1[1:], ages2[:-1])
            
            # Print results
            print(f"\nSequences {i} and {i+1}:")
            print(f"Driver: {seq1['DriverNumber'].iloc[0]}, Stint: {seq1['Stint'].iloc[0]}, Compound: {seq1['CompoundID'].iloc[0]}")
            print(f"Tire ages seq {i}: {ages1}")
            print(f"TARGET values seq {i}: {target1}")
            print(f"Expected target ages seq {i}: {expected_target_ages1}")
            print(f"Tire ages seq {i+1}: {ages2}")
            print(f"TARGET values seq {i+1}: {target2}")
            print(f"Expected target ages seq {i+1}: {expected_target_ages2}")
            print(f"Sliding window pattern: {sliding_window_correct}")
            
            # Verify that target1 corresponds to the next values after seq1
            # We'd need the original dataframe to check this precisely
            
            # Only check a limited number
            num_to_check -= 1
            if num_to_check <= 0:
                break
    
    print("\nVERIFICATION SUMMARY:")
    print("1. Each sequence should advance by one lap (sliding window pattern)")
    print("2. Targets should contain the FuelAdjustedDegPercent values for the next 3 laps")
    print("3. Each target should start exactly where its sequence ends")

In [ ]:
# Verirfy sequences with their targets
verify_sequences_with_targets(sequences, targets)

### Validation of Data Sequencing
#### Sliding Window Pattern:
- Each sequence advances exactly one lap relative to the previous one.
- Example: `[1,2,3,4,5] → [2,3,4,5,6] → [3,4,5,6,7] → [4,5,6,7,8]`

#### Consistency in Targets:
- Targets always represent the next 3 laps after each sequence.
- Example: Sequence 0 ends at lap 5, targets are laps 6, 7, 8.

#### Coherence Between Sequences and Targets:
- The target values also "slide" accordingly:
  - **Target of Sequence 0:** `[-3.88, -3.73, -4.36]`
  - **Target of Sequence 1:** `[-3.73, -4.36, -3.83]`
- The first two values of Target 1 match the last two of Target 0.

#### Maintaining Structure by Driver, Stint, and Compound:
- All shown sequences belong to the same driver (1), same stint (1.0), and same compound (2).
- This ensures we are analyzing the degradation of a single set of tires.

### Implications for Our LSTM Model
- The model can learn degradation patterns based on consecutive lap sequences.
- It can predict degradation for the next 3 laps using the previous 5 laps.
- The data structure captures both the absolute degradation level and its rate of change.

With this verification, we confirm that the data is correctly prepared for training the LSTM model. We have selected `FuelAdjustedDegPercent` as our target, which is appropriate since it represents degradation adjusted for fuel effects, precisely what we aim to predict

---

## 15 LSTM: Data Preparation

In [ ]:
def prepare_for_lstm(sequences, targets):
    """
    Convert the list of DataFrames and targets into numpy arrays suitable for LSTM training
    """
    # Get the number of features (columns) in the sequence DataFrames
    n_features = len(sequences[0].columns)
    sequence_length = len(sequences[0])
    prediction_horizon = len(targets[0])
    
    # Initialize arrays
    X = np.zeros((len(sequences), sequence_length, n_features))
    y = np.zeros((len(sequences), prediction_horizon))
    
    # Fill the arrays
    for i, (seq, target) in enumerate(zip(sequences, targets)):
        X[i] = seq.values
        y[i] = target
    
    print(f"Prepared data for LSTM with shape: X: {X.shape}, y: {y.shape}")
    return X, y


In [ ]:
# Prepare data for LSTM (convert to numpy arrays)
X, y = prepare_for_lstm(sequences, targets)

### 15.1 **Prepared Data:**
- **X**: (763, 5, 16) → 763 sequences, each containing 5 laps and 16 features per lap
- **y**: (763, 3) → For each sequence, we predict 3 future laps of degradation

---

### 15.2 **Dataset Split:**

In [ ]:
# Split data into train, validation and test sets (70-15-15)
# First split: separate test set (15%)
from sklearn.model_selection import train_test_split


X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42
)

# Second split: divide remaining data into train (70%) and validation (15%)
# The validation should be 17.65% of the temporary set (0.15/0.85)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.15/0.85, random_state=42
)

---

In [ ]:
# Print the shapes of the resulting datasets
print("Data split:")
print(f"X_train shape: {X_train.shape} ({len(X_train)/len(X):.1%})")
print(f"X_val shape: {X_val.shape} ({len(X_val)/len(X):.1%})")
print(f"X_test shape: {X_test.shape} ({len(X_test)/len(X):.1%})")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"y_test shape: {y_test.shape}")

- **Train**: 533 sequences (69.9%) - very close to the 70% target
- **Validation**: 115 sequences (15.1%) - very close to the 15% target
- **Test**: 115 sequences (15.1%) - very close to the 15% target

---

## 16. Pytorch Data Preparation

### 16.1 Creating Tensors

In [ ]:
# Train
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
# Validation
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val)
# Test
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)



### 16.2 Creating Datasets 

In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)


### 16.3 Creating Dataloaders

In [ ]:
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size= BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

### 16.3 Prepare Data by Compound

In [ ]:
def prepare_compound_data(compound_sequences, compound_targets):
    """
    
    Prepare data of each compound for the model
    
    """
    
    compound_data = {}
    
    for compound in unique_compounds:
        if len(compound_sequences[compound]) > 0:
            # Convert sequences to numpy format
            X, y = prepare_for_lstm(compound_sequences[compound], compound_targets[compound])
            compound_data[compound] = (X, y)
            
            print(f"Compound {compound_names.get(compound, compound)}: X shape {X.shape}, y shape {y.shape}")
    
    return compound_data



In [ ]:
# Prepare specific data for each tire
compound_data = prepare_compound_data(compound_sequences, compound_targets)

---

## 17. LSTM Architecture: Changed to Temporal Convolutional Network

In [ ]:
# class TireDegradationLSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, output_size, dropout = 0.3):
#         super(TireDegradationLSTM, self).__init__()

#         self.hidden_size = hidden_size
#         self.num_layers = num_layers

#         # LSTM Layer for Capturing sequential patterns
#         # RUN 2 : Bidirectional lstm for camputiring more patterns

#         self.lstm = nn.LSTM(
#             input_size= input_size,
#             hidden_size= hidden_size,
#             num_layers=num_layers,
#             batch_first= True,
#             dropout= dropout if num_layers > 1 else 0 ,
#             bidirectional= True # With this parameter it becomes bidirecional

#         )

#         # Layer normalization for stabilizing training
#         self.layer_norm1 = nn.LayerNorm(hidden_size * 2)  # *2 because it is bidirectional
        
#         # RUN 2: deeper dense layer system with redidual conections
#         self.fc1 = nn.Linear(hidden_size * 2, hidden_size)
#         self.layer_norm2 = nn.LayerNorm(hidden_size)
#         self.dropout1 = nn.Dropout(dropout)
        
#         self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
#         self.layer_norm3 = nn.LayerNorm(hidden_size // 2)
#         self.dropout2 = nn.Dropout(dropout)
        
#         self.fc3 = nn.Linear(hidden_size // 2, output_size)
        
#         # Activations
#         self.relu = nn.ReLU()

#     def forward(self, x):

#         # x shape must be (batch_size, sequence_length, input_size)

#         # LSTM output
#         lstm_out, _ = self.lstm(x)

#         # Use only last time step output
#         lstm_out = lstm_out[:, -1, :]

#         # RUN 2 Normalization and First FC layer
#         lstm_out = self.layer_norm1(lstm_out)

#         # First dense layer with normalization and residual
#         out = self.fc1(lstm_out)
#         out = self.layer_norm2(out)
#         out = self.relu(out)
#         out = self.dropout1(out)

#         # Second dense layer
#         residual = out  # We store it for residual conection
#         out = self.fc2(out)
#         out = self.layer_norm3(out)
#         out = self.relu(out)
#         out = self.dropout2(out)

#         # Output layer

#         out = self.fc3(out)
        

#         return out

In [ ]:
import torch.nn.functional as F

In [ ]:
class EnhancedTCN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=64, dropout=0.3):
        super(EnhancedTCN, self).__init__()
        
        # 1. Increase capacity and add regularization
        # Project the input to a higher-dimensional space using a 1D convolution.
        self.input_proj = nn.Conv1d(input_size, hidden_size, kernel_size=1)
        # Batch normalization on the projected input.
        self.bn_input = nn.BatchNorm1d(hidden_size)
        
        # 2. Multi-Scale Block with Exponential Dilations
        # Residual layers with different dilation rates to capture multi-scale patterns.
        # First dilated convolution with dilation=1.
        self.dilated_conv1 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding='same', dilation=1)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        # Second dilated convolution with dilation=2.
        self.dilated_conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding='same', dilation=2)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        # Third dilated convolution with dilation=4.
        self.dilated_conv4 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding='same', dilation=4)
        self.bn4 = nn.BatchNorm1d(hidden_size)
        # Fourth dilated convolution with dilation=8.
        self.dilated_conv8 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding='same', dilation=8)
        self.bn8 = nn.BatchNorm1d(hidden_size)
        
        # 3. Simple Temporal Attention Mechanism
        # Use a 1D convolution to compute attention scores over the sequence.
        self.attention = nn.Conv1d(hidden_size, 1, kernel_size=1)
        # Softmax is applied along the time dimension (seq_len).
        self.softmax = nn.Softmax(dim=2)
        
        # 4. Output Layer with Higher Dropout for Regularization
        self.dropout = nn.Dropout(dropout)
        # Fully connected layers for final prediction.
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, output_size)
        
    def forward(self, x):
        # x: [batch_size, seq_len, features]
        # Transpose the input to [batch, features, seq_len] for Conv1d operations.
        x = x.transpose(1, 2)
        
        # Initial projection: apply convolution, batch norm and ReLU activation.
        x = F.relu(self.bn_input(self.input_proj(x)))
        
        # Apply dilated convolutions with residual connections.
        # First residual block with dilation=1.
        residual = x
        x = F.relu(self.bn1(self.dilated_conv1(x)))
        x = x + residual  # Residual connection
        
        # Second residual block with dilation=2.
        residual = x
        x = F.relu(self.bn2(self.dilated_conv2(x)))
        x = x + residual  # Residual connection
        
        # Third residual block with dilation=4.
        residual = x
        x = F.relu(self.bn4(self.dilated_conv4(x)))
        x = x + residual  # Residual connection
        
        # Fourth residual block with dilation=8.
        residual = x
        x = F.relu(self.bn8(self.dilated_conv8(x)))
        x = x + residual  # Residual connection
        
        # Apply the attention mechanism.
        # Compute attention weights for each time step.
        attn_weights = self.softmax(self.attention(x))  # Shape: [batch, 1, seq_len]
        # Multiply feature maps with the attention weights.
        x = x * attn_weights
        
        # Global pooling: sum over the time dimension.
        x = torch.sum(x, dim=2)  # Resulting shape: [batch, channels]
        x = self.dropout(x)
        
        # Pass through fully connected layers with dropout and ReLU activations.
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x


---

### 17.1. Hyperparameters


In [ ]:
# ################################ HYPERPARAMETER DEFINITION #######################################

# ### RUN 2 
# input_size = 16 # Number of features
# hidden_size = 128  # Hidden layer size: from 64 to 128
# num_layers = 3 # Number of LSTM layers: from 2 to 3
# output_size = 3 # Number of future laps to predict
# learning_rate = 0.0005 # Reduced from 0.001 for more stable training
# weight_decay = 1e-5  # Added l2 regularization


# num_epochs = 150 # From 100, more for allowing convergence with new architecture
# patience = 20 # Apply early stopping. If no improvements after 15 epochs, stop training. Augmented to 20 for more convergence


In [ ]:
############################################### TCN HYPERPARAMETERS ##################################
input_size = 16
output_size = 3
learning_rate = 0.0001  # Lower learning rate
weight_decay = 1e-6
batch_size = 32
num_epochs = 200
patience = 25



#### 17.1.1. Hyperparameters for specific compound models

In [ ]:
# Shorter training cicles for specific models
compound_epochs = 100  
compound_patience = 15

---

### 17.2 Initializing Model

In [ ]:
######################################### LSTM ###################################
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")          #
# model = TireDegradationLSTM(input_size, hidden_size, num_layers, output_size)  #
# model.to(device)                                                               #
##################################################################################

In [ ]:
####################################### TCN ######################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EnhancedTCN(input_size, output_size)
model.to(device)

---

### 17.3 Loss Function and Optimizer

In [ ]:
#from torch.optim.lr_scheduler import ReduceLROnPlateau
# criterion = nn.MSELoss()
# # RUN 2: added L2 regularization
# optimizer = optim.Adam(model.parameters(), lr= learning_rate, weight_decay= weight_decay)

# #RUN 2 : learning rate programmer
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=7, verbose=True)

In [ ]:
########################### TCN #####################################


optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=0.0005,  #  Initial higher learning rate
    weight_decay=1e-4  # Stronger regularization
)
# Scheduler with cicled warming
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, 
    T_0=5,  # Initial 5 epoch cicling
    T_mult=2,  # Duplicate cicle in each restart
    eta_min=1e-6  # Minimum learning rate value
)
criterion = nn.MSELoss()

---

## 18. Training

### 18.1 Training Function for a single Epoch

#### 18.1.1 LSTM Train Epoch Commented

In [ ]:
####################################### LSTM ####################################
# def train_epoch(model, train_loader, criterion, optimizer, device):
#     model.train()
#     running_loss = 0.0
#
#     for inputs, targets in train_loader:
#         inputs, targets = inputs.to(device), targets.to(device)
#
#         # Forward pass
#
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#
#         # Backpropagation and Optimization
#
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#
#        
#
#         running_loss += loss.item() * inputs.size(0)
#
#     epoch_loss = running_loss / len(train_loader.dataset)
#     return epoch_loss
####################################################################################

        

---

#### 18.1.2 Train Epoch for TCN

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backpropagation y optimización
        optimizer.zero_grad()
        loss.backward()
        # Añadir gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

### 18.1 Evaluation Function

In [ ]:
def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item() * inputs.size(0)

    total_loss = running_loss / len(data_loader.dataset)
    return total_loss

---

## 19. Training with Early Stopping

#### 19.1 LSTM Training Loop

In [ ]:
# best_val_loss = float("inf")
# counter = 0 # Counter for epochs without improvement
# train_losses = []
# val_losses = []


# print("Starting Training...")

# for epoch in range(num_epochs):
#     train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
#     val_loss = evaluate(model, val_loader, criterion, device)


#     train_losses.append(train_loss)
#     val_losses.append(val_loss)

#     print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')

#     # Check early stopping condition

#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         counter = 0
#         # Store best model

#         torch.save(model.state_dict(), "../../outputs/week5/models/tire_degradation_lstm.pth")
#         print(f" Saving model (val_loss improved: {best_val_loss:.6f})")

#     else:
#         counter += 1
#         if counter >= patience:
#             print(f"Early stopping on epoch {epoch + 1}")
#             break

#### 19.2 TCN Training Loop

In [ ]:
# Main loop with early stopping and proper model saving

# Initialize best validation loss as infinity for tracking improvement
best_val_loss = float('inf')
counter = 0  # Counter for early stopping
train_losses = []  # Store training loss per epoch
val_losses = []  # Store validation loss per epoch

print("Starting Training...")
for epoch in range(num_epochs):
    # Train for one epoch and get the training loss
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    # Evaluate on the validation set and get the validation loss
    val_loss = evaluate(model, val_loader, criterion, device)
    
    # Store losses for later analysis
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    # Learning rate scheduler updates the LR based on validation loss
    scheduler.step(val_loss)
    
    # Print training progress
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')
    
    # Early stopping and model saving logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss  # Update best validation loss
        counter = 0  # Reset early stopping counter
        # Save the model in a DIFFERENT file than the LSTM model
        torch.save(model.state_dict(), '../../outputs/week5/models/tire_degradation_tcn.pth')
        print(f"Saving model (improved val_loss: {best_val_loss:.6f})")
    else:
        counter += 1  # Increment early stopping counter
        if counter >= patience:  # If no improvement for 'patience' epochs
            print(f"Early stopping at epoch {epoch+1}")
            break  # Stop training


### 19.3 Training models by Compound

In [ ]:
# Cell 19.a: Function to train models for each compound
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_compound_model(compound_id, X, y, device):
    """Trains a specific model for a given compound"""
    
    # Split data into train, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    # Create datasets
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    
    # Dataloaders
    compound_train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    compound_val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    compound_test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    
    # Initialize model
    compound_model = EnhancedTCN(input_size, output_size)
    compound_model.to(device)
    
    # Optimizer and scheduler
    optimizer = torch.optim.AdamW(
        compound_model.parameters(), 
        lr=0.0005,
        weight_decay=1e-4
    )
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, 
        T_0=5,  # Initial restart period (5 epochs)
        T_mult=2,  # Multiplier for increasing restart periods
        eta_min=1e-6  # Minimum learning rate
    )
    
    criterion = nn.MSELoss()  # Mean Squared Error Loss function
    
    # Training with early stopping
    best_val_loss = float('inf')  # Initialize best validation loss as infinity
    counter = 0  # Early stopping counter
    train_losses = []  # Store training losses per epoch
    val_losses = []  # Store validation losses per epoch
    
    for epoch in range(compound_epochs):
        # Train for one epoch and compute training loss
        train_loss = train_epoch(compound_model, compound_train_loader, criterion, optimizer, device)
        # Evaluate on validation set and compute validation loss
        val_loss = evaluate(compound_model, compound_val_loader, criterion, device)
        
        # Store loss values
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # Update learning rate scheduler
        scheduler.step()
        
        print(f'Epoch {epoch+1}/{compound_epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')
        
        # Early stopping and model saving logic
        if val_loss < best_val_loss:
            best_val_loss = val_loss  # Update best validation loss
            counter = 0  # Reset early stopping counter
            # Save the model with a unique filename per compound
            torch.save(compound_model.state_dict(), f'../../outputs/week5/models/tcn_compound_{compound_id}.pth')
            print(f"Saving model (improved val_loss: {best_val_loss:.6f})")
        else:
            counter += 1  # Increment early stopping counter
            if counter >= compound_patience:  # If no improvement for 'compound_patience' epochs
                print(f"Early stopping at epoch {epoch+1}")
                break  # Stop training early
    
    # Load the best model version before testing
    compound_model.load_state_dict(torch.load(f'../../outputs/week5/models/tcn_compound_{compound_id}.pth'))
    
    # Evaluate on test set
    test_loss = evaluate(compound_model, compound_test_loader, criterion, device)
    
    # Model evaluation metrics
    compound_model.eval()  # Set model to evaluation mode
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for inputs, targets in compound_test_loader:
            inputs = inputs.to(device)
            outputs = compound_model(inputs)
            predictions.append(outputs.cpu().numpy())
            actuals.append(targets.numpy())
    
    if len(predictions) > 0:
        # Convert lists into arrays for metric calculation
        predictions = np.vstack(predictions)
        actuals = np.vstack(actuals)
        
        mse = mean_squared_error(actuals, predictions)
        rmse = np.sqrt(mse)  # Root Mean Squared Error
        mae = mean_absolute_error(actuals, predictions)
        
        metrics = {
            'mse': mse,
            'rmse': rmse,
            'mae': mae
        }
    else:
        # Handle case where no predictions are generated
        metrics = {'mse': float('nan'), 'rmse': float('nan'), 'mae': float('nan')}
    
    return compound_model, metrics, (train_losses, val_losses)


#### Training Loop per Compound

In [ ]:
specialized_models = {}
compound_performance = {}
compound_loss_curves = {}

for compound in compound_data:
    compound_name = compound_names.get(compound, f"Compound {compound}")
    print(f"\n=== Training model for {compound_name} ===")
    
    X, y = compound_data[compound]
    
    # Only train if enough data
    if len(X) > 10:  
        model, metrics, loss_curves = train_compound_model(compound, X, y, device)
        
        specialized_models[compound] = model
        compound_performance[compound] = metrics
        compound_loss_curves[compound] = loss_curves
    else:
        print(f"Not enough data for {compound_name} tire")

#### Obtain Combined Learning Rates

In [ ]:
# Extract learning curves for plotting
train_losses_combined = []
val_losses_combined = []

# Determine the maximum length among all learning curves
max_length = 0
for compound in compound_loss_curves:
    train_curve, val_curve = compound_loss_curves[compound]
    max_length = max(max_length, len(train_curve))

# Compute the average of the learning curves
for i in range(max_length):
    train_sum = 0
    val_sum = 0
    count = 0
    
    for compound in compound_loss_curves:
        train_curve, val_curve = compound_loss_curves[compound]
        if i < len(train_curve):
            train_sum += train_curve[i]
            val_sum += val_curve[i]
            count += 1
    
    if count > 0:
        train_losses_combined.append(train_sum / count)
        val_losses_combined.append(val_sum / count)

# Use these combined curves for plotting
train_losses = train_losses_combined
val_losses = val_losses_combined


---

## 20. Load Best Model

In [ ]:
#model.load_state_dict(torch.load("../../outputs/week5/models/tire_degradation_lstm.pth"))
model.load_state_dict(torch.load('../../outputs/week5/models/tire_degradation_tcn.pth'))

---

## 21. Evaluating on test Set

In [ ]:
test_loss = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.6f}")

In [ ]:
## Making predictions on test set
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        predictions.append(outputs.cpu().numpy())
        actuals.append(targets.numpy())

predictions = np.vstack(predictions)
actuals = np.vstack(actuals)


---

## 22. Calculating metrics

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)

In [ ]:
print(f"Test MSE: {mse:.6f}")
print(f"Test RMSE: {rmse:.6f}")
print(f"Test MAE: {mae:.6f}")

### 22.1 Losses in training and validation

In [ ]:
# Plotting training and validation losses
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.savefig('../../outputs/week5/training_validation_loss.png')
plt.show()

### 22.2 Predictions vs Real values

In [ ]:
plt.figure(figsize=(14, 7))

# Select some samples for visualizing
sample_indices = np.random.choice(len(predictions), size=5, replace=False)

for i, idx in enumerate(sample_indices):
    plt.subplot(2, 3, i+1)
    
    horizon = range(1, 4)  # 3 future laps
    
    plt.plot(horizon, actuals[idx], 'o-', label='Actual', color='blue')
    plt.plot(horizon, predictions[idx], 'o--', label='Predicted', color='red')
    
    plt.title(f'Sample {idx}')
    plt.xlabel('Future Lap')
    plt.ylabel('Degradation (s/lap)')
    plt.grid(True)
    if i == 0:
        plt.legend()

plt.tight_layout()
plt.savefig('../../outputs/week5/predictions_vs_actual.png')
plt.show()

---

### 22.3 Performance Comparison by Compound

In [ ]:
# Display table header for metrics per compound
print("\nPerformance by Compound:")
# Format the header with columns for Compound, MSE, RMSE, and MAE
print("{:<15} {:<12} {:<12} {:<12}".format("Compound", "MSE", "RMSE", "MAE"))
print("-" * 55)  # Print a separator line

# Create a dictionary for the global metrics from previously computed values
global_metrics = {"mse": mse, "rmse": rmse, "mae": mae}

# Loop through each compound and its corresponding performance metrics
for compound, metrics in compound_performance.items():
    # Retrieve the compound name from the dictionary, or default to "Compound <id>"
    compound_name = compound_names.get(compound, f"Compound {compound}")
    
    # Print the metrics for the current compound, formatted with 6 decimal places
    print("{:<15} {:<12.6f} {:<12.6f} {:<12.6f}".format(
        compound_name, 
        metrics['mse'], 
        metrics['rmse'], 
        metrics['mae']
    ))

# Add the global model metrics for comparison, using the same formatting
print("{:<15} {:<12.6f} {:<12.6f} {:<12.6f}".format(
    "GLOBAL MODEL", 
    global_metrics['mse'], 
    global_metrics['rmse'], 
    global_metrics['mae']
))

# Create a bar chart to visually compare the RMSE of each compound model against the global model

# Set the size of the figure
plt.figure(figsize=(12, 6))

# Extract the compound identifiers from the performance dictionary keys
compounds = list(compound_performance.keys())
# Create a list of RMSE values for each compound from the performance metrics
rmse_values = [metrics['rmse'] for metrics in compound_performance.values()]
# Create a list of compound labels using the compound_names dictionary, defaulting if not found
compound_labels = [compound_names.get(c, f"Compound {c}") for c in compounds]

# Define colors for each compound using a dictionary, defaulting to 'gray' if the compound is not specified
colors = [compound_colors.get(c, 'gray') for c in compounds]

# Plot a bar chart where each bar corresponds to the RMSE value of a compound model
plt.bar(compound_labels, rmse_values, color=colors)

# Draw a horizontal dashed red line representing the global model's RMSE for easy comparison
plt.axhline(y=rmse, color='red', linestyle='--', label=f'Global Model (RMSE={rmse:.3f})')

# Label the y-axis with a description and the measurement units (seconds/round)
plt.ylabel('RMSE (seconds/round)')
# Set the title of the chart to describe the comparison being made
plt.title('RMSE Comparison by Tire Type')
# Add a legend to differentiate between the global model and compound-specific models
plt.legend()
# Enable grid lines for the y-axis to improve readability
plt.grid(axis='y', alpha=0.3)

# Save the figure to the specified output path
plt.savefig('../../outputs/week5/compound_specialized_performance.png')
# Display the plot
plt.show()


---

### 22.4 Visualizations of Predictions vs Actual Values by Compound

In [ ]:
def visualize_compound_predictions(compound_id, compound_model, X_test, y_test, device):
    """
    Visualizes predictions versus actual values for a specific compound.
    """
    # Filter test data indices corresponding to this compound.
    compound_test_indices = []
    for i, sequence in enumerate(sequences):
        # Check if the current sequence belongs to the given compound using the 'CompoundID'
        if sequence['CompoundID'].iloc[0] == compound_id:
            compound_test_indices.append(i)
    
    # Convert the test data to a PyTorch tensor and send it to the specified device (CPU or GPU).
    X_compound = torch.FloatTensor(X_test).to(device)
    
    # Set the model to evaluation mode and compute predictions without tracking gradients.
    compound_model.eval()
    with torch.no_grad():
        y_pred = compound_model(X_compound).cpu().numpy()
    
    # y_actual remains the provided ground truth values for comparison.
    y_actual = y_test
    
    # Determine the number of samples to visualize (up to 5 samples, or less if fewer predictions exist)
    n_samples = min(5, len(y_pred))
    if n_samples > 0:
        # Randomly select sample indices without replacement.
        sample_indices = np.random.choice(len(y_pred), size=n_samples, replace=False)
        
        # Create a figure with a grid of subplots (2 rows and 3 columns) to display each sample.
        fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(14, 8))
        axes = axes.flatten()  # Flatten the 2D array of axes to a 1D list for easier indexing.
        
        # Retrieve the compound name from the dictionary, or default to "Compound <compound_id>".
        compound_name = compound_names.get(compound_id, f"Compound {compound_id}")
        fig.suptitle(f'Predictions vs. Actual Values for {compound_name}', fontsize=16)
        
        # Plot each selected sample in a separate subplot.
        for i, idx in enumerate(sample_indices):
            if i < len(axes):
                ax = axes[i]
                
                # Define the forecast horizon; here, we assume predictions for 3 future laps.
                horizon = range(1, 4)  # Future laps: 1, 2, 3
                
                # Plot the actual values as a blue solid line with markers.
                ax.plot(horizon, y_actual[idx], 'o-', label='Actual', color='blue')
                # Plot the predicted values as a red dashed line with markers.
                ax.plot(horizon, y_pred[idx], 'o--', label='Predicted', color='red')
                
                # Set subplot title, labels, and enable grid lines.
                ax.set_title(f'Sample {idx}')
                ax.set_xlabel('Future Lap')
                ax.set_ylabel('Degradation (s/lap)')
                ax.grid(True)
                
                # Only add a legend to the first subplot.
                if i == 0:
                    ax.legend()
        
        # Turn off any unused subplots if less than the available axes.
        for i in range(n_samples, len(axes)):
            axes[i].axis('off')
        
        # Adjust the layout to accommodate the figure title and subplots.
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        # Save the figure to the specified output path, including the compound name in the filename.
        plt.savefig(f'../../outputs/week5/predictions_{compound_name}.png')
        plt.show()
    else:
        print(f"There are not enough samples to visualize for compound {compound_name}")

# Visualize predictions for each compound in the specialized models.
for compound in specialized_models:
    # Retrieve the compound name using the compound_names dictionary.
    compound_name = compound_names.get(compound, f"Compound {compound}")
    print(f"\nVisualizing predictions for {compound_name}...")
    
    # Filter the test data indices for this compound.
    compound_indices_test = []
    for i, seq in enumerate(sequences):
        # Ensure that the index is within the bounds of the test data and matches the compound.
        if i < len(X_test) and seq['CompoundID'].iloc[0] == compound:
            compound_indices_test.append(i)
    
    # If there are test samples for this compound, proceed with visualization.
    if len(compound_indices_test) > 0:
        # Filter the test data (both features and labels) using the collected indices.
        X_compound_test = X_test[compound_indices_test]
        y_compound_test = y_test[compound_indices_test]
        
        # Call the visualization function with the filtered data.
        visualize_compound_predictions(compound, specialized_models[compound], 
                                      X_compound_test, y_compound_test, device)
    else:
        print(f"There is no test data for compound {compound_name}")


---

### 22.5 Compare Learning Curves by Compound

In [ ]:
plt.figure(figsize=(12, 7))  # Set the size of the figure

# Loop through each compound's loss curves data
for compound in compound_loss_curves:
    # Retrieve the compound name from the dictionary, defaulting to "Compound <id>" if not found
    compound_name = compound_names.get(compound, f"Compound {compound}")
    # Get the color for the compound; default to black if not specified
    color = compound_colors.get(compound, 'black')
    
    # Each entry in compound_loss_curves is a tuple: (training loss curve, validation loss curve)
    train_curve, val_curve = compound_loss_curves[compound]
    
    # Plot the training loss curve using a dashed line with some transparency
    plt.plot(train_curve, '--', color=color, alpha=0.7, 
             label=f'{compound_name} (Train)')
    # Plot the validation loss curve using a solid line
    plt.plot(val_curve, '-', color=color, 
             label=f'{compound_name} (Val)')

# Set the x-axis label to "Epoch"
plt.xlabel('Epoch')
# Set the y-axis label to "Loss"
plt.ylabel('Loss')
# Set the title for the plot
plt.title('Learning Curves by Tire Type')
# Display a legend to differentiate between compounds and data types (training vs validation)
plt.legend()
# Enable grid lines for better readability
plt.grid(True)
# Save the figure to the specified output path
plt.savefig('../../outputs/week5/learning_curves_by_compound.png')
# Display the plot
plt.show()


---

## 23. Ensemble System for Combined Predictions

In [ ]:
def ensemble_predict(X, compound_id, global_model, specialized_models, device, 
                     weights=None):
    """
    Makes predictions using an ensemble of the global model and the specialized model.
    
    Args:
        X: Input data (tensor)
        compound_id: ID of the compound for which the prediction is made
        global_model: Global model trained on all data
        specialized_models: Dictionary containing specialized models for each compound
        device: Device for computation (CPU/GPU)
        weights: Weights to combine predictions [global_weight, specialized_weight]
                 If None, weights are calculated based on the inverse RMSE.
    
    Returns:
        Ensemble predictions and the weights used for combining
    """
    # Convert X to a tensor if it is not already one.
    if not isinstance(X, torch.Tensor):
        X = torch.FloatTensor(X)
    
    # Move the input tensor to the specified device (CPU/GPU)
    X = X.to(device)
    
    # Get predictions from the global model.
    # Set the model to evaluation mode and disable gradient computation.
    global_model.eval()
    with torch.no_grad():
        global_pred = global_model(X).cpu().numpy()
    
    # Check if there is a specialized model for the given compound.
    if compound_id in specialized_models:
        specialized_model = specialized_models[compound_id]
        specialized_model.eval()
        with torch.no_grad():
            specialized_pred = specialized_model(X).cpu().numpy()
            
        # If weights are not provided, calculate them based on the inverse of the RMSE.
        if weights is None:
            global_rmse = 0.355017  # RMSE of the global model
            specialized_rmse = compound_performance[compound_id]['rmse']
            
            # Use the inverse of the RMSE as the weighting factor (better performance = higher weight)
            global_weight = 1 / global_rmse
            specialized_weight = 1 / specialized_rmse
            
            # Normalize the weights so that their sum is equal to 1.
            total = global_weight + specialized_weight
            global_weight /= total
            specialized_weight /= total
            
            weights = [global_weight, specialized_weight]
        
        # Combine the predictions from the global and specialized models using the calculated weights.
        ensemble_pred = weights[0] * global_pred + weights[1] * specialized_pred
        
        return ensemble_pred, weights
    else:
        # If no specialized model exists, return only the global prediction.
        return global_pred, [1.0, 0.0]

# Evaluate the performance of the ensemble system.
ensemble_predictions = []      # List to store ensemble predictions for each compound.
ensemble_weights_used = {}     # Dictionary to store average weights used for each compound.

print("\nEvaluating Ensemble System:")
test_indices_by_compound = {}  # Dictionary to group test indices by compound.

# Group test indices by compound.
for i in range(len(X_test)):
    # Get the compound associated with the i-th test example.
    # We need to link it with the original sequence.
    test_sequence_idx = i % len(sequences)  # Approximation; adjust if necessary.
    if test_sequence_idx < len(sequences):
        compound_id = sequences[test_sequence_idx]['CompoundID'].iloc[0]
        
        if compound_id not in test_indices_by_compound:
            test_indices_by_compound[compound_id] = []
        
        test_indices_by_compound[compound_id].append(i)

# Evaluate the ensemble for each compound.
for compound_id, indices in test_indices_by_compound.items():
    # Retrieve compound name from dictionary; default if not found.
    compound_name = compound_names.get(compound_id, f"Compound {compound_id}")
    # Filter the test data (features and labels) for the current compound.
    X_compound = X_test[indices]
    y_compound = y_test[indices]
    
    compound_predictions = []  # List to store predictions for this compound.
    weights_sum = [0, 0]       # To accumulate the weights used over all samples.
    
    # Iterate through each test sample for the current compound.
    for i, x in enumerate(X_compound):
        # Reshape the sample to have a batch dimension of 1.
        x_tensor = torch.FloatTensor(x.reshape(1, x.shape[0], x.shape[1]))
        # Get the ensemble prediction and the weights used for this sample.
        pred, weights = ensemble_predict(x_tensor, compound_id, model, 
                                         specialized_models, device)
        compound_predictions.append(pred[0])
        # Sum the weights for averaging later.
        weights_sum[0] += weights[0]
        weights_sum[1] += weights[1]
    
    # Calculate average weights used across all samples for the compound.
    if len(X_compound) > 0:
        avg_weights = [w / len(X_compound) for w in weights_sum]
        ensemble_weights_used[compound_id] = avg_weights
    
    # Convert the predictions list to a NumPy array.
    compound_predictions = np.array(compound_predictions)
    
    # Calculate evaluation metrics: Mean Squared Error, RMSE, and Mean Absolute Error.
    mse = mean_squared_error(y_compound, compound_predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_compound, compound_predictions)
    
    # Retrieve the RMSE of the global and specialized models.
    global_rmse = 0.355017
    specialized_rmse = compound_performance.get(compound_id, {'rmse': float('inf')})['rmse']
    
    # Calculate the percentage improvement of the ensemble over the global and specialized models.
    improvement_over_global = (global_rmse - rmse) / global_rmse * 100
    improvement_over_specialized = (specialized_rmse - rmse) / specialized_rmse * 100
    
    # Print performance metrics for the current compound.
    print(f"\n{compound_name}:")
    print(f"  Ensemble RMSE: {rmse:.6f}")
    print(f"  Global RMSE: {global_rmse:.6f} ({improvement_over_global:.2f}% improvement)")
    print(f"  Specialized RMSE: {specialized_rmse:.6f} ({improvement_over_specialized:.2f}% improvement)")
    print(f"  Weights used: Global={avg_weights[0]:.2f}, Specialized={avg_weights[1]:.2f}")
    
    # Save predictions for visualization.
    ensemble_predictions.append((compound_id, compound_predictions, y_compound))


---

## Visualization of Ensemble Predictions

In [ ]:
def visualize_ensemble_predictions(ensemble_results):
    """
    Visualizes the ensemble model's predictions against the actual values.
    
    Args:
        ensemble_results: List of tuples (compound_id, predictions, actuals)
    """
    # Iterate over each compound's ensemble results
    for compound_id, predictions, actuals in ensemble_results:
        # Retrieve the compound name from the dictionary, or default if not found
        compound_name = compound_names.get(compound_id, f"Compound {compound_id}")
        
        # Select up to 5 samples to visualize
        n_samples = min(5, len(predictions))
        if n_samples == 0:
            continue  # Skip if there are no samples
        
        # Randomly choose sample indices without replacement
        sample_indices = np.random.choice(len(predictions), size=n_samples, replace=False)
        
        # Create a figure with a grid of subplots (2 rows x 3 columns)
        fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(14, 8))
        axes = axes.flatten()  # Flatten to simplify indexing
        
        # Set the overall title for the figure
        fig.suptitle(f'Ensemble Predictions vs. Actual Values for {compound_name}', fontsize=16)
        
        # Loop over each selected sample index to plot
        for i, idx in enumerate(sample_indices):
            if i < len(axes):
                ax = axes[i]
                
                # Define the forecast horizon (here, 3 future laps)
                horizon = range(1, 4)
                
                # Prepare the input sample for prediction by reshaping and sending to device
                # Assuming each sample has a shape that should be reshaped to (1, 5, 16)
                x_tensor = torch.FloatTensor(X_test[idx].reshape(1, 5, 16)).to(device)
                
                # Obtain the global model prediction for the sample
                with torch.no_grad():
                    global_pred = model(x_tensor).cpu().numpy()[0]
                
                # Check if there is a specialized model for this compound; if so, use it
                if compound_id in specialized_models:
                    specialized_model = specialized_models[compound_id]
                    with torch.no_grad():
                        specialized_pred = specialized_model(x_tensor).cpu().numpy()[0]
                else:
                    specialized_pred = global_pred  # Fallback to global prediction
                
                # Plot the actual values as a blue line with markers
                ax.plot(horizon, actuals[idx], 'o-', label='Actual', color='blue')
                # Plot the ensemble prediction as a purple dashed line with markers
                ax.plot(horizon, predictions[idx], 'o--', label='Ensemble', color='purple')
                # Plot the global model prediction as a red dashed line with a different marker, semi-transparent
                ax.plot(horizon, global_pred, 'x--', label='Global', color='red', alpha=0.4)
                # Plot the specialized model prediction as a green dashed line with a different marker, semi-transparent
                ax.plot(horizon, specialized_pred, '*--', label='Specialized', color='green', alpha=0.4)
                
                # Set subplot title and axis labels
                ax.set_title(f'Sample {idx}')
                ax.set_xlabel('Future Lap')
                ax.set_ylabel('Degradation (s/lap)')
                ax.grid(True)
                
                # Only add a legend in the first subplot for clarity
                if i == 0:
                    ax.legend()
        
        # Hide any unused subplots in the grid
        for i in range(n_samples, len(axes)):
            axes[i].axis('off')
        
        # Adjust layout to ensure the overall title does not overlap with subplots
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        # Save the figure to the specified file path
        plt.savefig(f'../../outputs/week5/ensemble_predictions_{compound_name}.png')
        # Display the plot
        plt.show()

# Visualize ensemble predictions using the results from the ensemble evaluation
visualize_ensemble_predictions(ensemble_predictions)


---

## 26. Making prediction function

For being able to use all these models fast and as part of a bigger system, it is necessary to implement a big function that uses all the steps defined in this notebook. 

Then, I will implement a new notebook, called `N02_model_tire_predictions`, where I will make this. This is important for the future to convert this notebook to a Python module that I will call in the logic agent for the rules related with degradation where the models will need to predict based on the available data.

---